In [4]:
import stanza
import preProcessing

nlp = stanza.Pipeline(lang='hi', processors='tokenize,pos,lemma', use_gpu=True)
data = 'नहीं वीडियो दिल्ली का सरकारी स्कूल मदरसा तब्दील नहीं दिखातायह केजरीवाल दिल्ली स्कूल मॉडल जो सेकुलर हिंदू लग केजरीवाल अच्छा काम देख ले दिल्ली का सरकारी स्कूल केजरीवाल मदरसा बदल शुरू दिल्ली का विजय नगर सरकारी स्कूल स्कूल कलमा उर्दू पढ़ इजाजत दे केजरीवाल बिधायको सरकार समर्थन हिंदू तय जिहादी सोच अरविंद केजरीवाल पार्टी का'

hindi_doc = nlp(data)
lemmas = [word.lemma for sent in hindi_doc.sentences for word in sent.words]

text = ' '.join(i for i in lemmas if i != None)

2022-06-07 10:47:22 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |

2022-06-07 10:47:22 INFO: Use device: cpu
2022-06-07 10:47:22 INFO: Loading: tokenize
2022-06-07 10:47:22 INFO: Loading: pos
2022-06-07 10:47:23 INFO: Loading: lemma
2022-06-07 10:47:23 INFO: Done loading processors!


In [3]:
text

'नहीं वीडियो दिल्ली का सरकारी स्कूल मदरसा तब्दील नहीं दिखा केजरीवाल दिल्ली स्कूल मॉडल जो सेकुलर हिंदू लग केजरीवाल अच्छा काम देख ले दिल्ली का सरकारी स्कूल केजरीवाल मदरसा बदल शुरू दिल्ली का विजय नगर सरकारी स्कूल स्कूल कलमा उर्दू पढ़ इजाजत दे केजरीवाल बिधायको सरकार समर्थन हिंदू तय जिहादी सोच अरविंद केजरीवाल पार्टी का'

In [5]:
path = 'google/muril-base-cased'
## Loading the model
from transformers import AutoModel, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path,output_hidden_states=True)
## Embeddings 
#text ="कोई अच्छी सी फिल्म लगायो"
#text = "दोस्त"
print(tokenizer.convert_ids_to_tokens(tokenizer.encode(text)))
input_encoded = tokenizer.encode_plus(text, return_tensors="pt")
with torch.no_grad():
         states = model(**input_encoded).hidden_states
output = torch.stack([states[i] for i in range(len(states))])
#output = output.squeeze()
print("Output shape is {}".format(output.shape))

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['[CLS]', 'नहीं', 'वीडियो', 'दिल्ली', 'का', 'सरकारी', 'स्कूल', 'मदरसा', 'तब्दील', 'नहीं', 'दिखा', 'केजरीवाल', 'दिल्ली', 'स्कूल', 'मॉडल', 'जो', 'से', '##कुलर', 'हिंदू', 'लग', 'केजरीवाल', 'अच्छा', 'काम', 'देख', 'ले', 'दिल्ली', 'का', 'सरकारी', 'स्कूल', 'केजरीवाल', 'मदरसा', 'बदल', 'शुरू', 'दिल्ली', 'का', 'विजय', 'नगर', 'सरकारी', 'स्कूल', 'स्कूल', 'कलम', '##ा', 'उर्दू', 'पढ़', 'इजाजत', 'दे', 'केजरीवाल', 'बि', '##धाय', '##को', 'सरकार', 'समर्थन', 'हिंदू', 'तय', 'जिहाद', '##ी', 'सोच', 'अरविंद', 'केजरीवाल', 'पार्टी', 'का', '[SEP]']
Output shape is torch.Size([13, 1, 62, 768])


In [9]:
token_vecs = output[-2][0]
print(token_vecs.shape)
sentence_embedding = torch.mean(token_vecs, dim=0)
print(sentence_embedding.shape)

torch.Size([62, 768])
torch.Size([768])


In [7]:
x = sentence_embedding.tolist()
print(type(x))

<class 'list'>


In [11]:
len(x)

768

In [16]:
import numpy as np
xx = np.array(x)
xx = xx.reshape(1, -1)
print(xx.shape)

(1, 768)


In [12]:
print(x[0], type(x[0]))

-0.04854370653629303 <class 'float'>


In [18]:
import keras

model = keras.models.load_model('modelCase.h5')
pred = model.predict(xx)
print(pred)

[[0.00107089]]
